ТЕОРИЈА

За да филтрираме слика во фреквенциски домен потребно е да ја помножиме сликата во фреквенциски домен со филтерот (кернел) кој е исто така
во фреквенциски домен.

\begin{equation}
I_{f, filt}= H_f
\end{equation}

За таа цел потребно е да се вметне Octave функција која ќе креира “Band pass
gaussian filter” во фреквенциски домен. Bandpass гаусовиот филтер во фреквенциски домен е дефиниран со равенката:

\begin{equation}
H(i, j)= e^\frac{-d(i,j)^2}{f^2_h}(1- e^\frac{-d(i,j)^2}{f^2_l})
\end{equation}

Каде што $d(i, j)$ е евклидовото растојанието на даден пиксел од центарот
на сликата, додека $f_h$ и $f_l$ се праговите на високите и ниските фреквенции кои сакаме да ги филтрираме.
Конверзија на 2D сликата од просторен во фреквенциски домен се прави со користење на Фуриева трансформација.
Фуриевата трансформација е дефинирана како:

\begin{equation}
F(u, v) = \iint_{-\infty}^{\infty}f(x,y) e^{-i2\pi(ux+vy)}dxdy
\end{equation}

А во дискретниот домен горната равенка може да биде преведена како

\begin{equation}
F(u, v) = \sum_{m=-\infty}^{\infty}\sum_{n=-\infty}^{\infty}f[m,n] e^{-i2\pi(umx_0+vny_0)}
\end{equation}

Конверзија на 2D сликата од фреквенциски во просторен домен се прави со користење на инверзна Фуриева трансформација. Инверзната Фуриева трансформација е дефинирана како:

\begin{equation}
f(x, y) = \iint_{-\infty}^{\infty}F(u, v) e^{i2\pi(ux+vy)}dudv
\end{equation}

А во дискретниот домен горната равенка може да биде преведена како

\begin{equation}
f(x, y) = \sum_{m=-\infty}^{\infty}\sum_{n=-\infty}^{\infty}F(m,n) e^{i2\pi(xmu_0+ynv_0)}
\end{equation}


In [1]:
%%file create_filter.m
function filter = create_filter(nx,ny,d0,d1)

filter = ones(nx,ny);

for i = 0:nx-1,
            for j = 0:ny-1
                 dist= sqrt((i-nx/2)^2 + (j-ny/2)^2);
                 filter(i+1,j+1) = exp(-dist^2/(2*(d1^2))).*(1.0-exp(-dist^2/(2*(d0^2))));
            end
end

filter(nx/2+1,ny/2+1)=1;
end

Created file 'C:\Users\user\Downloads\create_filter.m'.


In [ ]:
clear all
%read image
im = double(imread('C:\Users\user\Barbara.tif'));
%size of the image
[nx ny] = size(im);
%Transform the image from spatial to frequency domain
im_fft = fftshift(fft2(im));
% Create Gaussian bandpass kernels, in frequency domain, for 20 different cut off frequencies and filter the image
p=1;
for i = 1:10:101
q=1;
    for j = 1:10:101
        %Create the bandpass kernel
        tmp1 = create_filter(nx,ny,i,j);
        %Filter the image with the bandpass kernel and transform it back to spatial domain
        tmp = abs(ifft2(ifftshift(tmp1.*im_fft)));
        %Save the filtered image (in spatial domain) and kernel (in frequency domain) for visualization 
        filtered_image(p,q,:) = tmp(:);
        filter_kernel(p,q,:) = tmp1(:);
        q=q+1;
    end
    p=p+1;
end

save filtered_image
save filter_kernel

In [ ]:
import ipywidgets as widgets
widgets.IntRangeSlider(
    value=[0, 10],
    min=0,
    max=10,
    step=1,
    description='Frueqeuncy cuttoffs:',
    style={'description_width': 'initial'} ,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',

)

In [ ]:
from scipy.io import loadmat
import matplotlib.pyplot as plt
import scipy.io as sio
import numpy as np

filtriranaslika = sio.loadmat('filtered_image.mat')
filtrirankernel = sio.loadmat('filter_kernel.mat')
imgfiltslika = filtered_image['filtered_image']
imgfiltkernel = filtered_kernel['filter_kernel']

def plot(value):
    image = imgfiltslika
    kernel =  imgfiltkernel
    plot_image = np.concatenate((np.fliplr(image), np.fliplr(kernel)), axis=1)
    plt.figure(figsize=(20,30))
    plt.imshow(plot_image, cmap="gray")
    plt.title('Filtered Image\n')
    plt.axis('off')
    plt.show()
    
widgets.interactive(plot, value=value)

